# Assignment 04: Single-source shortest path


---

### Codebase


In [2]:
# fmt: off

def sssp(s, G):
    n: int = len(G)                         # Shortcut to size of graph
    no_edge = G[0][0]                       # Shortcut to absence of edge
    oo = float("inf")                       # Shortcut to infinity
    d: list = [oo for _ in range(n)]        # Assume looooong distances
    d[s] = 0                                # Source distance to itself
    bag = [s]                               # Start from source
    while len(bag) > 0:                     # While bag not empty
        u = bag.pop()                       # Take a vertex out of bag
        for v in range(n):                  # Find u's neighbors
            if G[u][v] != no_edge:          # There is an edge (u,v)
                if d[v] > d[u] + G[u][v]:   # Is edge (u,v) tense?
                    d[v] = d[u] + G[u][v]   # Relax the edge
                    bag.append(v)           # Explore v next
    return d                                # Shortest path distances



---

The code above is far from perfect but it works as long as there are no cycles in the graph.


### Part 1: reconstruct the shortest paths

The code above produces a list `d` with the shortest path distances from a source vertex $s$ to any vertex in the graph.

Write a method with the following header

```python
reconstruct(d: list[int], s: int, graph: list[list[int]]) -> list[int]
```

that returns a list of `p` such that `p[u]` is the predecessor of `u` in the shortedt path from the source vertex (`s`) to `u`. 

**In practice,** methods `sssp` and `reconstruct_path` are combined into a single method. Such a method is given at the end of this solutions notebook.

In [3]:
def reconstruct_path(d: list[int], s: int, graph: list[list[int]]) -> list[int]:
    # Shortcut to size of graph
    n: int = len(graph)
    # Shortcut to infinity 
    oo: int = float("inf")
    # Shortcut to no edge
    no_edge: int = graph[0][0]
    # Initialize list of predecessors to None (no predecessor)
    pred: list[int | None] = [None for _ in range(n)]
    # Traverse vertices in order of index and trace back
    # their predecessors.
    v: int = 0
    while v < n:
        # No need to trace back source vertex
        if v != s: 
            # Find all the neighbors of v; one of them is
            # the predecessor of v along the shortest path from s to v.
            # Once that predecessor is found, the search of v'set
            # neighbors can stop.
            u: int = 0
            while u < n and pred[v] is None:
                # Variable for the weight of edge (u,v) makes code readable
                weight: int = graph[u][v]
                # check that edge exists in the input graph and
                # that u is readhable from s(i.e., d[u] != oo) before 
                # checking if u is v's predecessor
                if weight != 0 and d[u] != oo:
                    # Check is edg (u,v) is relaxed; 
                    # if it is, then u is v's predecessor
                    if d[v] == d[u] + weight:
                        pred[v] = u
                # Increment u to check next neighbor of v for inner loop
                u += 1
        # Increment v to check next vertex for outer loop
        v += 1
    # Done
    return pred





### Part 2: report about the shortest paths

Write a method with header

```python
report_sssp(p: list[int], d: list[int], graph: list[list[int]]) -> None
```

that prints the shortest path to every vertex in the graph, from the source vertex. For example, the shortest path to vertex 7 is `0 -> 2 -> 3 -> 5 -> 6 -> 7`



In [4]:
def report_sssp(p: list[int], d: list[int], graph: list[list[int]]) -> None:
    NO_EDGE: int = graph[0][0]
    NO_PATH: str = "No path exists from {} to {}."
    SOURCE_VERTEX: str = "{} is the source vertex"
    oo:int = float("inf")
    n: int = len(graph)
    # Find the source vertex first, by looking for the vertex with no predecessor
    s: int = 0
    while s < n and p[s] is not None:
        s += 1
    print(f"source vertex is {s}")
    # Exlore each vertex v in order of index and report the path from s to v
    for v in range(n):
        if v== s:
            # Special handling for source vertex
            print(SOURCE_VERTEX.format(s))
        else:
            # For the rest of the vertices, first we need to determine
            # if there is a path from s to v by checking if d[v] is infinity
            # if there is no path, report that fact; otherwise, trace back the path
            if d[v] == oo:
                print(NO_PATH.format(s, v))
            else:
                # Traceback the path from s to v using the list of predecessors p
                path: list[int] = []
                u: int = v
                while u != s:
                    path.append(u)
                    u = p[u]
                path.append(s)
                path.reverse()
                print(f"path from {s} to {v}: {path}")


---

# Test

![](https://raw.githubusercontent.com/lgreco/images/refs/heads/main/graphs/dag_sssp.png)


In [5]:
# fmt: off
graph = [
    #0   1   2   3   4   5   6   7
    [0,  5,  1,  5, 10,  0,  0,  0],  # 0
    [0,  0, 12,  5,  6,  0,  0,  0],  # 1
    [0,  0,  0,  1,  0,  0,  5,  0],  # 2
    [0,  0,  0,  0,  0,  1,  5,  0],  # 3
    [0,  0,  0,  6,  0,  5,  0,  5],  # 4
    [0,  0,  0,  0,  0,  0,  1,  5],  # 5
    [0,  0,  0,  0,  0,  0,  0,  1],  # 6
    [0,  0,  0,  0,  0,  0,  0,  0],  # 7
]

source = 0
sp_distances = sssp(source, graph)
print(f"{sp_distances=}")
sp_predecessors = reconstruct_path(sp_distances, source, graph)
print(f"{sp_predecessors=}")
report_sssp(sp_predecessors, sp_distances, graph)

sp_distances=[0, 5, 1, 2, 10, 3, 4, 5]
sp_predecessors=[None, 0, 0, 2, 0, 3, 5, 6]
source vertex is 0
0 is the source vertex
path from 0 to 1: [0, 1]
path from 0 to 2: [0, 2]
path from 0 to 3: [0, 2, 3]
path from 0 to 4: [0, 4]
path from 0 to 5: [0, 2, 3, 5]
path from 0 to 6: [0, 2, 3, 5, 6]
path from 0 to 7: [0, 2, 3, 5, 6, 7]



---

# Everything in a single method

Typically, the SSSP is implemented as a single method computing both shortest path distances and predecessors for each vertix -- except, of course, the source vertex. The purpose of this exercise is to demonstrate a useful characteristic of *dynamic programming:* such algorithms solve problems in two parts, like we did in this assignment. Fortunately, the SSSP can be done in a single pass, as shown below. The modifications are shown with comments marked with `#####`.


In [ ]:
# fmt: off

def sssp_combined(s, G):
    n: int = len(G)                         # Shortcut to size of graph
    no_edge = G[0][0]                       # Shortcut to absence of edge
    oo = float("inf")                       # Shortcut to infinity
    d: list = [oo for _ in range(n)]        # Assume looooong distances
    d[s] = 0                                # Source distance to itself
    p: list = [None for _ in range(n)]      ##### Assume no predecessors
    bag = [s]                               # Start from source
    while len(bag) > 0:                     # While bag not empty
        u = bag.pop()                       # Take a vertex out of bag
        for v in range(n):                  # Find u's neighbors
            if G[u][v] != no_edge:          # There is an edge (u,v)
                if d[v] > d[u] + G[u][v]:   # Is edge (u,v) tense?
                    d[v] = d[u] + G[u][v]   # Relax the edge
                    p[v] = u                ##### Record predecessor of v
                    bag.append(v)           # Explore v next
    return d, p                             ##### Return distances and predecessors

print(sssp_combined(source, graph))

([0, 5, 1, 2, 10, 3, 4, 5], [None, 0, 0, 2, 0, 3, 5, 6])
